# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'/content/drive/My Drive/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [0]:
# !pip uninstall scikit-learn

Uninstalling scikit-learn-0.19.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scikit_learn-0.19.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-0.19.1


In [3]:
 !pip install scikit-learn==0.20.3
 

     |████████████████████████████████| 5.4MB 9.2MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [1]:
import sklearn
sklearn.__version__

'0.20.3'

In [0]:
import pickle
import joblib

path = '/content/drive/My Drive/'
infile = open(path +'sentiment_36.pkl', 'rb' )
modelo = joblib.load(infile)

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [3]:
!pip install numpy
!pip install scipy
!pip install --upgrade watson-developer-cloud
!pip install --upgrade watson-machine-learning-client
!pip install --upgrade "ibm-watson>=4.3.0"

     |████████████████████████████████| 256kB 9.1MB/s 
     |████████████████████████████████| 204kB 53.1MB/s 
  Created wheel for watson-developer-cloud: filename=watson_developer_cloud-2.10.1-cp36-none-any.whl size=252813 sha256=fb4da6fbbf94f339f084c381632e85c36074afa3f931815c42b31d1e08355393
  Stored in directory: /root/.cache/pip/wheels/ea/a7/19/fac0a408a586265eb374005308a7553d4494ab41b2fd88f5bc
Successfully built watson-developer-cloud
     |████████████████████████████████| 542kB 9.4MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 768kB 42.3MB/s 
     |████████████████████████████████| 225kB 66.1MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.6.0-py2.py3-none-any.whl size=72547 sha256=16bf84b8fc795a792b6c4a9c8e7c5e7339e14aa346e4a41b2f92762173fa5bb0
  Stored in directory: /root/.cache/pip/wheels/37/9c/c4/a2c610ccb877d37c2cb87a5bfe55845fecffd6bb01bcd5e9d5
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core

In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [2]:
wml_credentials={
   "apikey": "*****************",
  "iam_apikey_description": "Auto-generated for key 2fd8d0e8-69a6-472b-aa96-7773925308e0",
  "iam_apikey_name": "Credenciales de servicio-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/0d3a87150c3f4773b2e56673c54364d5::serviceid:ServiceId-b35007de-9f2f-4e29-a096-3d2e0ec546a3",
  "instance_id": "a7252000-1ea4-4b6d-a89a-ac778d724426",
  "url": "https://us-south.ml.cloud.ibm.com"
}

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [0]:
client = WatsonMachineLearningAPIClient(wml_credentials)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [0]:
model_properties = {client.repository.ModelMetaNames.AUTHOR_NAME: "Emilio_Chambouleyron", 
               client.repository.ModelMetaNames.NAME: "movie_reviews_tfdif_7",
               client.repository.DefinitionMetaNames.RUNTIME_NAME: 'python',
               client.repository.DefinitionMetaNames.RUNTIME_VERSION: '3.6'}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode',
                             lowercase = True,
                             stop_words='english',
                            token_pattern='[A-Za-z]+')

pipeline = make_pipeline(vectorizer, modelo)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth...001,
     verbose=0))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [0]:
published_model_nlp = client.repository.store_model(model=pipeline, 
                                                meta_props=model_properties, 
                                                training_data=X_train, 
                                                training_target=y_train)


**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [0]:
published_model_nlp_uid = client.repository.get_model_uid(published_model_nlp)
nlp_model_details = client.repository.get_details(published_model_nlp_uid)

In [14]:
models_details = client.repository.list_models()

------------------------------------  ---------------------  ------------------------  -----------------
GUID                                  NAME                   CREATED                   FRAMEWORK
c7c1ac83-90dd-4ccf-b63f-1badf282bc70  movie_reviews_tfdif_7  2020-03-20T19:18:24.818Z  scikit-learn-0.20
37978d9d-1698-4a32-9952-437f63f7a77d  movie_reviews_tfdif_7  2020-03-19T15:30:21.589Z  scikit-learn-0.19
a4531196-dc22-48ad-b560-1c783957905a  movie_reviews_tfdif_7  2020-03-18T13:13:26.714Z  scikit-learn-0.19
------------------------------------  ---------------------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [0]:
loaded_model = client.repository.load(published_model_nlp_uid)

In [16]:
loaded_model

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth...001,
     verbose=0))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None))])

In [0]:
test_predictions = loaded_model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [18]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print("roc_auc score: ", roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

roc_auc score:  0.8575
              precision    recall  f1-score   support

           0       0.83      0.91      0.86       200
           1       0.90      0.81      0.85       200

   micro avg       0.86      0.86      0.86       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.86      0.86      0.86       400

